## BB example

### helper functions

In [2]:
import gzip
from collections import defaultdict

def readJSON(path):
  for l in gzip.open(path, 'rt'):
    d = eval(l)
    u = d['userID']
    try:
      g = d['gameID']
    except Exception as e:
      g = None
    yield u,g,d

### define paths

In [7]:
"""
==================================================================================
TRAINING DATA (train.json.gz)
==================================================================================
DESCRIPTION: 175,000 instances to be used for training. This data should be used 
for both the play prediction and time played prediction tasks. It is not necessary 
to use all observations for training, for example if doing so proves too 
computationally intensive.
    userID The ID of the user. 
    gameID The ID of the game. 
    text Text of the user's review of the game.
    date Date when the review was entered.
    hours How many hours the user played the game.
    hours transformed log_{2}(hours+1). 
        !Note!: log_{2}(hours+1) = hours_transformed
        * This transformed value is the one we are trying to predict
__________________________________________________________________________________
==================================================================================
GOAL 1: predict hours_transformed given userID and gameID
==================================================================================
DESCRIPTION:Predict how long a person will play a game (transformed as log2
(hours + 1), for those (user,game) pairs in pairs Hours.csv. Accuracy will be measured in terms of the mean-squared
error (MSE).
    !Note!: log_{2}(hours+1) = hours_transformed)
subset (pairs_Hours.csv):
    userID,gameID,prediction
    u04763917,g51093074
    u10668484,g42523222
    u82502949,g39422502
    u14336188,g83517324
    u10096161,g10962300
    u12864301,g29951417
    u74352605,g88258978
    u75043948,g25417282
    u29296791,g59132435
__________________________________________________________________________________
==================================================================================    
GOAL 2: predict 'played' given userID and gameID
==================================================================================

DESCRIPTION: Predict given a (user,game) pair from pairs Played.csv whether the 
user would play the game (0 or 1). Accuracy will be measured in terms of the 
categorization accuracy (fraction of correct predictions). 
    !Note!:The test set has been constructed such that exactly 50% of the pairs 
    correspond to played games and the other 50% do not.
subset (pairs_Played.csv):
    userID,gameID,prediction
    u04836696,g41031307
    u32377855,g62450068
    u58289072,g71021765
    u74685029,g26732871
    u06266052,g69433247
    u45011836,g57175884
    u98484614,g01435414
    u34253509,g66197269
    u76252009,g10053132
__________________________________________________________________________________
"""
# define data paths
train_json = '/home/scotty/dsc_256/fall_25/make_up/assignment1/train.json.gz'
pairs_hours = '/home/scotty/dsc_256/fall_25/make_up/assignment1/pairs_Hours.csv'
pairs_played = '/home/scotty/dsc_256/fall_25/make_up/assignment1/pairs_Played.csv'

In [12]:
# create containers for users and games
user_dict = defaultdict(list)
game_dict = defaultdict(list)
train_data = []

# read train.json.gz and populate user_dict and game_dict
for u,g,d in readJSON(train_json):
    user_dict[u].append(g)
    game_dict[g].append(u)
    train_data.append(d)

In [14]:
print(user_dict['u04836696'])
print(train_data[1])

['g05339526', 'g26654626', 'g56751692', 'g88534444', 'g35155801', 'g68132896', 'g70072143', 'g09480895', 'g02141122', 'g46446145', 'g29741733', 'g84367501', 'g04462740', 'g95765068', 'g83425645', 'g47086763', 'g97784906', 'g36222633', 'g34147651', 'g27545941', 'g13121437', 'g30645652', 'g42862258', 'g54979645', 'g34397868', 'g34842774', 'g64509433', 'g14120436', 'g40372626', 'g79333823', 'g68302537', 'g71073909', 'g66258359', 'g12460730', 'g83595955', 'g46430686', 'g43436077', 'g48254339', 'g84227207', 'g01269364', 'g07027969', 'g66296428', 'g53666531', 'g67071738', 'g16087475', 'g15881340', 'g88681493', 'g10773791', 'g03162218', 'g32641915', 'g90153689', 'g87795702', 'g51259924', 'g77859458', 'g24741744', 'g41141819', 'g33400185', 'g16097342', 'g62438733']
{'userID': 'u70666506', 'early_access': False, 'hours': 63.5, 'hours_transformed': 6.011227255423254, 'found_funny': 1, 'text': 'If you want to sit in queue for 10-20min and have 140 ping then this game is perfect for you :)', 'game

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np 


def softplus(x, beta=1.0):
    return (1/beta) * np.log(1 + np.exp(beta * x))
df = pd.DataFrame(train_data)
df['text_len_norm'] = df['text'].str.len()/df['text'].str.len().max()
df['user_idx'], user_ids = pd.factorize(df['userID'])
df['item_idx'], item_ids = pd.factorize(df['gameID'])

num_users = len(user_ids)
num_items = len(item_ids)
# Calculate thresholds once (more efficient than inside the loop)
hours_threshold = df['hours_transformed'].mean()
text_threshold = df['text_len_norm'].mean()

def create_monotonic_chain(row):
    """
    Creates a ground truth vector y that enforces y_1 >= y_2 >= y_3
    """
    # Stage 1: Played (Always 1 for this dataset)
    y1 = 1
    
    # Stage 2: Engaged (High Hours)
    y2 = 1 if row['hours_transformed'] > hours_threshold else 0
    
    # Stage 3: Reviewed (Long Text)
    y3 = 1 if row['text_len_norm'] > text_threshold else 0
    
    # --- CRITICAL FIX: Enforce Monotonicity ---
    # If they reviewed (y3=1), they MUST be considered engaged (y2=1)
    # Logic: "A review action implies... a 'purchase' action" [cite: 9]
    if y3 == 1:
        y2 = 1
        
    return [y1, y2, y3]

# Apply the function
# Rename 'intention' to 'chain_labels' or 'y_true' to avoid confusion.
# "Intention" usually refers to the model's *output* score (delta), not the input data.
df['chain_labels'] = df.apply(create_monotonic_chain, axis=1)

# Quick check: The sum of the chain should tell you the "Edge" (l*)
# [1, 0, 0] -> Sum 1 -> Edge 1
# [1, 1, 0] -> Sum 2 -> Edge 2
# [1, 1, 1] -> Sum 3 -> Edge 3
df['edge_index'] = df['chain_labels'].apply(sum)

print(df[['hours_transformed', 'text_len_norm', 'chain_labels', 'edge_index']].head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175000 entries, 0 to 174999
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   hours              175000 non-null  float64
 1   gameID             175000 non-null  object 
 2   hours_transformed  175000 non-null  float64
 3   early_access       175000 non-null  bool   
 4   date               175000 non-null  object 
 5   text               175000 non-null  object 
 6   userID             175000 non-null  object 
 7   found_funny        29938 non-null   float64
 8   user_id            54841 non-null   object 
 9   compensation       2870 non-null    object 
 10  text_len_norm      175000 non-null  float64
dtypes: bool(1), float64(4), object(6)
memory usage: 13.5+ MB
None
               hours  hours_transformed   found_funny  text_len_norm
count  175000.000000      175000.000000  29938.000000  175000.000000
mean       66.408189           3.717845      7.

### all

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

############################################################
###########        READ/PREP TRAINING DATA       ###########
############################################################
# read data
df = pd.DataFrame(train_data)

# define features: normailized text length, user_idx(userID -> int), item_idx(gameID -> int)
df['text_len_norm'] = df['text'].str.len()/df['text'].str.len().max()
df['user_idx'], user_ids = pd.factorize(df['userID'])
df['item_idx'], item_ids = pd.factorize(df['gameID'])

# get user/game counts
num_users = len(user_ids)
num_items = len(item_ids)
# calc threshhold for transfrmed hours and normalized text length
hours_threshold = df['hours_transformed'].mean()
text_threshold = df['text_len_norm'].mean()

# define create_monotonic_chain function
def create_monotonic_chain(row):
    """
    Creates a ground truth vector y that enforces y_1 >= y_2 >= y_3
    """
    y1 = 1 # Played
    y2 = 1 if row['hours_transformed'] > hours_threshold else 0 # Engaged
    y3 = 1 if row['text_len_norm'] > text_threshold else 0      # Reviewed
    
    # Enforce Monotonicity: Review -> Engaged
    if y3 == 1:
        y2 = 1
        
    return [y1, y2, y3]

# define features: chain_labels and edge_index
df['chain_labels'] = df.apply(create_monotonic_chain, axis=1)
df['edge_index'] = df['chain_labels'].apply(sum) # 1, 2, or 3

print(f"Data Prepared. Users: {num_users}, Items: {num_items}")
print(df[['userID', 'edge_index']].head())
############################################################


############################################################
#######    DEFINE CHAINREC MODEL AND EDGE LOSS     #########
############################################################
class ChainRec(nn.Module):
    def __init__(self, num_users, num_items, num_stages=3, embed_dim=16, beta=1.0):
        super().__init__()
        self.num_stages = num_stages
        self.beta = beta
        
        # 1. Embeddings (User & Item)
        self.user_emb = nn.Embedding(num_users, embed_dim)
        self.item_emb = nn.Embedding(num_items, embed_dim)
        
        # 2. Biases
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_bias = nn.Embedding(num_items, 1)
        self.global_bias = nn.Parameter(torch.zeros(1))
        
        # 3. Stage Embeddings (The "Lenses")
        # Shape: (num_stages, embed_dim)
        self.stage_emb = nn.Embedding(num_stages, embed_dim)
        
        # Initialize weights (Optional but good practice)
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)
        nn.init.xavier_uniform_(self.stage_emb.weight)

    def forward(self, u_idx, i_idx):
        """
        Returns: A tensor of shape (batch_size, num_stages) 
                 containing scores [s_1, s_2, s_3] for each pair.
        """
        # Look up embeddings
        u = self.user_emb(u_idx)      # (batch, dim)
        i = self.item_emb(i_idx)      # (batch, dim)
        
        # Biases
        b_u = self.user_bias(u_idx).squeeze()
        b_i = self.item_bias(i_idx).squeeze()
        base_score = self.global_bias + b_u + b_i
        
        # Interaction vector (Element-wise product) [cite: 310]
        interaction = u * i           # (batch, dim)
        
        # Calculate Intention (delta) for each stage
        # We want delta_l = dot(stage_l, interaction)
        # We can do this via matrix multiplication for all stages at once
        # stages weight shape: (num_stages, dim)
        # interaction shape: (batch, dim)
        # result shape: (batch, num_stages)
        deltas = torch.matmul(interaction, self.stage_emb.weight.t())
        
        # Rectify Intentions (Softplus) [cite: 313]
        # delta_plus = (1/beta) * log(1 + exp(beta * delta))
        deltas_plus = F.softplus(deltas, beta=self.beta)
        
        # Cumulative Sum (Monotonicity Enforcement) [cite: 335]
        # We sum from l to L. 
        # In pytorch, flip -> cumsum -> flip achieves this "reverse cumsum"
        scores = torch.flip(torch.cumsum(torch.flip(deltas_plus, [1]), dim=1), [1])
        
        # Add base biases to every stage's score
        # scores shape: (batch, stages)
        # base_score shape: (batch) -> unsqueeze to (batch, 1)
        final_scores = scores + base_score.unsqueeze(1)
        
        return final_scores
def edge_loss(model, u_idx, i_idx_pos, edge_indices, i_idx_neg):
    """
    u_idx: User Indices
    i_idx_pos: The game they actually played
    edge_indices: Where they stopped (1, 2, or 3) - derived from your dataframe
    i_idx_neg: A random game they didn't play (Stage 0)
    """
    
    # 1. Get scores for Positive Items
    # Shape: (batch, 3) -> [s1, s2, s3]
    pos_scores = model(u_idx, i_idx_pos)
    
    # Select the score at the specific "edge" for each user
    # We need to use gather to pick s1 for user A, s3 for user B, etc.
    # edge_indices need to be 0-based for array indexing (Stage 1 -> index 0)
    # Your df['edge_index'] is likely 1, 2, 3. Subtract 1.
    gather_indices = (edge_indices - 1).unsqueeze(1) # Shape (batch, 1)
    s_edge_pos = pos_scores.gather(1, gather_indices).squeeze()
    
    # 2. Get scores for Negative Items
    neg_scores = model(u_idx, i_idx_neg)
    
    # For negative items, we assume the user is at Stage 0 (no interaction).
    # So we want to minimize the probability of entering Stage 1.
    # We look at the score for Stage 1 (index 0)
    s_neg = neg_scores[:, 0] 
    
    # 3. Compute Probabilities (Sigmoid)
    p_pos = torch.sigmoid(s_edge_pos)
    p_neg = torch.sigmoid(s_neg)
    
    # 4. Log Likelihood Loss [cite: 380]
    # Maximize log(p_pos) + log(1 - p_neg)
    # Which is equivalent to minimizing:
    loss = -torch.mean(torch.log(p_pos + 1e-10) + torch.log(1 - p_neg + 1e-10))
    
    return loss

############################################################

############################################################
##############         TRAINING LOOP         ###############
############################################################
# Prepare Tensors
train_u = torch.LongTensor(df['user_idx'].values)
train_i = torch.LongTensor(df['item_idx'].values)
train_edge = torch.LongTensor(df['edge_index'].values)

dataset = TensorDataset(train_u, train_i, train_edge)
# Small batch size for this tiny fake dataset
dataloader = DataLoader(dataset, batch_size=4, shuffle=True) 

model = ChainRec(num_users, num_items, num_stages=3)
optimizer = optim.Adam(model.parameters(), lr=0.005) # High LR for tiny data

print("\nStarting Training...")
model.train()
for epoch in range(10): # 10 Epochs
    total_loss = 0
    for b_u, b_i, b_edge in dataloader:
        optimizer.zero_grad()
        
        # Negative Sampling: Random item for each user
        b_neg_i = torch.randint(0, num_items, (len(b_u),))
        
        loss = edge_loss(model, b_u, b_i, b_edge, b_neg_i)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if epoch % 2 == 0:
        print(f"Epoch {epoch}: Loss {total_loss:.4f}")
############################################################


############################################################
########       PREDICTION AND REGRESSION LOGIC       #######
############################################################
# Create lookup maps for test data
u_map = {u: i for i, u in enumerate(user_ids)}
i_map = {i: idx for idx, i in enumerate(item_ids)}

test_play_df = pd.read_csv('pairs_Played.csv')
# 3. Use Maps to Translate Test Data
test_play_df['u_idx'] = test_play_df['userID'].map(lambda x: u_map.get(x, 0))
test_play_df['i_idx'] = test_play_df['gameID'].map(lambda x: i_map.get(x, 0))

# A. Predict "Played" (Stage 1 Score)

u_test = torch.LongTensor(test_play_df['u_idx'])
i_test = torch.LongTensor(test_play_df['i_idx'])

model.eval()
with torch.no_grad():
    scores = model(u_test, i_test)
    probs_played = torch.sigmoid(scores[:, 0]).numpy() # Stage 1

test_play_df['prediction'] = probs_played
print("\nPlayed Predictions:")
print(test_play_df)

test_hours_df = pd.read_csv('pairs_Hours.csv')
# 3. Use Maps to Translate Test Data
test_hours_df['u_idx'] = test_hours_df['userID'].map(lambda x: u_map.get(x, 0))
test_hours_df['i_idx'] = test_hours_df['gameID'].map(lambda x: i_map.get(x, 0))

# A. Predict "Played" (Stage 1 Score)

u_test = torch.LongTensor(test_hours_df['u_idx'])
i_test = torch.LongTensor(test_hours_df['i_idx'])

# B. Predict "Hours" (Regression from Scores)
# 1. Get scores for training data
with torch.no_grad():
    train_scores = model(train_u, train_i).numpy()

# 2. Train Regressor (Model Scores -> Real Hours)
regressor = LinearRegression()
regressor.fit(train_scores, df['hours_transformed'])

# 3. Predict
with torch.no_grad():
    test_scores = model(u_test, i_test).numpy()
    pred_hours = regressor.predict(test_scores)

test_hours_df['prediction'] = pred_hours
print("\nHours Predictions:")
print(test_hours_df)
############################################################

Data Prepared. Users: 6710, Items: 2437
      userID  edge_index
0  u55351001           1
1  u70666506           2
2  u18612571           1
3  u34283088           1
4  u16220374           1

Starting Training...
Epoch 0: Loss 569881.8742
Epoch 2: Loss 589084.5074
Epoch 4: Loss 586710.0344
Epoch 6: Loss 589763.4476


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

############################################################
###########        READ/PREP TRAINING DATA       ###########
############################################################
# read data
df = pd.DataFrame(train_data)

# define features: normailized text length, user_idx(userID -> int), item_idx(gameID -> int)
df['text_len_norm'] = df['text'].str.len()/df['text'].str.len().max()
df['user_idx'], user_ids = pd.factorize(df['userID'])
df['item_idx'], item_ids = pd.factorize(df['gameID'])

# get user/game counts
num_users = len(user_ids)
num_items = len(item_ids)
# calc threshhold for transfrmed hours and normalized text length
hours_threshold = df['hours_transformed'].mean()
text_threshold = df['text_len_norm'].mean()

# define create_monotonic_chain function
def create_monotonic_chain(row):
    """
    Creates a ground truth vector y that enforces y_1 >= y_2 >= y_3
    """
    y1 = 1 # Played
    y2 = 1 if row['hours_transformed'] > hours_threshold else 0 # Engaged
    y3 = 1 if row['text_len_norm'] > text_threshold else 0      # Reviewed
    
    # Enforce Monotonicity: Review -> Engaged
    if y3 == 1:
        y2 = 1
        
    return [y1, y2, y3]

# define features: chain_labels and edge_index
df['chain_labels'] = df.apply(create_monotonic_chain, axis=1)
df['edge_index'] = df['chain_labels'].apply(sum) # 1, 2, or 3

print(f"Data Prepared. Users: {num_users}, Items: {num_items}")
print(df[['userID', 'edge_index']].head())
############################################################


############################################################
#######    DEFINE CHAINREC MODEL AND EDGE LOSS     #########
############################################################
class ChainRec(nn.Module):
    def __init__(self, num_users, num_items, num_stages=3, embed_dim=16, beta=1.0):
        super().__init__()
        self.num_stages = num_stages
        self.beta = beta
        
        # 1. Embeddings (User & Item)
        self.user_emb = nn.Embedding(num_users, embed_dim)
        self.item_emb = nn.Embedding(num_items, embed_dim)
        
        # 2. Biases
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_bias = nn.Embedding(num_items, 1)
        self.global_bias = nn.Parameter(torch.zeros(1))
        
        # 3. Stage Embeddings (The "Lenses")
        # Shape: (num_stages, embed_dim)
        self.stage_emb = nn.Embedding(num_stages, embed_dim)
        
        # Initialize weights (Optional but good practice)
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)
        nn.init.xavier_uniform_(self.stage_emb.weight)

    def forward(self, u_idx, i_idx):
        """
        Returns: A tensor of shape (batch_size, num_stages) 
                 containing scores [s_1, s_2, s_3] for each pair.
        """
        # Look up embeddings
        u = self.user_emb(u_idx)      # (batch, dim)
        i = self.item_emb(i_idx)      # (batch, dim)
        
        # Biases
        b_u = self.user_bias(u_idx).squeeze()
        b_i = self.item_bias(i_idx).squeeze()
        base_score = self.global_bias + b_u + b_i
        
        # Interaction vector (Element-wise product) [cite: 310]
        interaction = u * i           # (batch, dim)
        
        # Calculate Intention (delta) for each stage
        # We want delta_l = dot(stage_l, interaction)
        # We can do this via matrix multiplication for all stages at once
        # stages weight shape: (num_stages, dim)
        # interaction shape: (batch, dim)
        # result shape: (batch, num_stages)
        deltas = torch.matmul(interaction, self.stage_emb.weight.t())
        
        # Rectify Intentions (Softplus) [cite: 313]
        # delta_plus = (1/beta) * log(1 + exp(beta * delta))
        deltas_plus = F.softplus(deltas, beta=self.beta)
        
        # Cumulative Sum (Monotonicity Enforcement) [cite: 335]
        # We sum from l to L. 
        # In pytorch, flip -> cumsum -> flip achieves this "reverse cumsum"
        scores = torch.flip(torch.cumsum(torch.flip(deltas_plus, [1]), dim=1), [1])
        
        # Add base biases to every stage's score
        # scores shape: (batch, stages)
        # base_score shape: (batch) -> unsqueeze to (batch, 1)
        final_scores = scores + base_score.unsqueeze(1)
        
        return final_scores
def edge_loss(model, u_idx, i_idx_pos, edge_indices, i_idx_neg):
    """
    u_idx: User Indices
    i_idx_pos: The game they actually played
    edge_indices: Where they stopped (1, 2, or 3) - derived from your dataframe
    i_idx_neg: A random game they didn't play (Stage 0)
    """
    
    # 1. Get scores for Positive Items
    # Shape: (batch, 3) -> [s1, s2, s3]
    pos_scores = model(u_idx, i_idx_pos)
    
    # Select the score at the specific "edge" for each user
    # We need to use gather to pick s1 for user A, s3 for user B, etc.
    # edge_indices need to be 0-based for array indexing (Stage 1 -> index 0)
    # Your df['edge_index'] is likely 1, 2, 3. Subtract 1.
    gather_indices = (edge_indices - 1).unsqueeze(1) # Shape (batch, 1)
    s_edge_pos = pos_scores.gather(1, gather_indices).squeeze()
    
    # 2. Get scores for Negative Items
    neg_scores = model(u_idx, i_idx_neg)
    
    # For negative items, we assume the user is at Stage 0 (no interaction).
    # So we want to minimize the probability of entering Stage 1.
    # We look at the score for Stage 1 (index 0)
    s_neg = neg_scores[:, 0] 
    
    # 3. Compute Probabilities (Sigmoid)
    p_pos = torch.sigmoid(s_edge_pos)
    p_neg = torch.sigmoid(s_neg)
    
    # 4. Log Likelihood Loss [cite: 380]
    # Maximize log(p_pos) + log(1 - p_neg)
    # Which is equivalent to minimizing:
    loss = -torch.mean(torch.log(p_pos + 1e-10) + torch.log(1 - p_neg + 1e-10))
    
    return loss

############################################################

############################################################
##############         TRAINING LOOP         ###############
############################################################
# Prepare Tensors
train_u = torch.LongTensor(df['user_idx'].values)
train_i = torch.LongTensor(df['item_idx'].values)
train_edge = torch.LongTensor(df['edge_index'].values)

dataset = TensorDataset(train_u, train_i, train_edge)
# Small batch size for this tiny fake dataset
dataloader = DataLoader(dataset, batch_size=4, shuffle=True) 

model = ChainRec(num_users, num_items, num_stages=3)
optimizer = optim.Adam(model.parameters(), lr=0.005) # High LR for tiny data

print("\nStarting Training...")
model.train()
for epoch in range(10): # 10 Epochs
    total_loss = 0
    for b_u, b_i, b_edge in dataloader:
        optimizer.zero_grad()
        
        # Negative Sampling: Random item for each user
        b_neg_i = torch.randint(0, num_items, (len(b_u),))
        
        loss = edge_loss(model, b_u, b_i, b_edge, b_neg_i)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if epoch % 2 == 0:
        print(f"Epoch {epoch}: Loss {total_loss:.4f}")
############################################################


############################################################
########       PREDICTION AND REGRESSION LOGIC       #######
############################################################
# Create lookup maps for test data
u_map = {u: i for i, u in enumerate(user_ids)}
i_map = {i: idx for idx, i in enumerate(item_ids)}

test_play_df = pd.read_csv('pairs_Played.csv')
# 3. Use Maps to Translate Test Data
test_play_df['u_idx'] = test_play_df['userID'].map(lambda x: u_map.get(x, 0))
test_play_df['i_idx'] = test_play_df['gameID'].map(lambda x: i_map.get(x, 0))

# A. Predict "Played" (Stage 1 Score)

u_test = torch.LongTensor(test_play_df['u_idx'])
i_test = torch.LongTensor(test_play_df['i_idx'])

model.eval()
with torch.no_grad():
    scores = model(u_test, i_test)
    probs_played = torch.sigmoid(scores[:, 0]).numpy() # Stage 1

test_play_df['prediction'] = probs_played
print("\nPlayed Predictions:")
print(test_play_df)

test_hours_df = pd.read_csv('pairs_Hours.csv')
# 3. Use Maps to Translate Test Data
test_hours_df['u_idx'] = test_hours_df['userID'].map(lambda x: u_map.get(x, 0))
test_hours_df['i_idx'] = test_hours_df['gameID'].map(lambda x: i_map.get(x, 0))

# A. Predict "Played" (Stage 1 Score)

u_test = torch.LongTensor(test_hours_df['u_idx'])
i_test = torch.LongTensor(test_hours_df['i_idx'])

# B. Predict "Hours" (Regression from Scores)
# 1. Get scores for training data
with torch.no_grad():
    train_scores = model(train_u, train_i).numpy()

# 2. Train Regressor (Model Scores -> Real Hours)
regressor = LinearRegression()
regressor.fit(train_scores, df['hours_transformed'])

# 3. Predict
with torch.no_grad():
    test_scores = model(u_test, i_test).numpy()
    pred_hours = regressor.predict(test_scores)

test_hours_df['prediction'] = pred_hours
print("\nHours Predictions:")
print(test_hours_df)
############################################################